<a href="https://colab.research.google.com/github/qngyn/BigData_Project1/blob/main/Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 48.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c295b378a3b4c86285874ddb7138caa1a457e797810ba5d22dd602884818a92f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
pip install pyodbc 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 472 kB 4.0 MB/s 


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyodbc 
import pandas as pd 
import pyspark.sql.functions as sf 

In [4]:
spark = SparkSession.builder.config('spark.driver.memory', '16g').getOrCreate()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
def process_log_data(path, file_name, date):
  df = spark.read.json(path+file_name)
  df = df.select('_source.AppName', '_source.Contract', '_source.Mac', '_source.TotalDuration')
  df = df.withColumn('Date', lit(date))
  df = df.withColumn('Type',
        when((col('AppName') == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
        .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
              (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
        .when((col("AppName") == 'RELAX'), "Giải Trí")
        .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
        .when((col("AppName") == 'SPORT'), "Thể Thao")
        .otherwise("Error"))
  df.select('Contract', 'Type', 'TotalDuration', 'Date')
  df.groupBy('Contract', 'Type', 'Date').agg({'TotalDuration':'sum'}).withColumnRenamed('sum(TotalDuration)', 'Total Durattion')
  return df

In [42]:
path ='/content/drive/MyDrive/Dataset/'
file_name='20220401.json'
date = '2022-04-01'
df = process_log_data(path, file_name, date)

In [40]:
df.show()

+-------+---------+------------+-------------+----------+-----------+
|AppName| Contract|         Mac|TotalDuration|      Date|       Type|
+-------+---------+------------+-------------+----------+-----------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|2022-04-01|Truyền Hình|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|2022-04-01|Truyền Hình|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|2022-04-01|Truyền Hình|
|  KPLUS|HND141717|08674EE8D2C2|         1452|2022-04-01|Truyền Hình|
|  KPLUS|HNH743103|402343C25D7D|          251|2022-04-01|Truyền Hình|
|  KPLUS|HNH893773|B84DEE76D3B8|          924|2022-04-01|Truyền Hình|
|  KPLUS|HND083642|B84DEE849A0F|         1444|2022-04-01|Truyền Hình|
|  KPLUS|DNFD74404|90324BB44C39|          691|2022-04-01|Truyền Hình|
|  KPLUS|DTFD21200|B84DEED27709|         1436|2022-04-01|Truyền Hình|
|  KPLUS|LDFD05747|0C96E6C95E53|         1434|2022-04-01|Truyền Hình|
|  KPLUS|HNH063566|B84DEEDD1C85|          687|2022-04-01|Truyền Hình|
|  KPLUS|HNH866786|1